In [1]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent.parent

if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

In [2]:
import importlib
import src.config as cfg
importlib.reload(cfg)

PROCESSED_DATA_DIR = cfg.PROCESSED_DATA_DIR
ZERO_TO_NAN_FILE = PROCESSED_DATA_DIR / 'zero_to_nan.csv'

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(ZERO_TO_NAN_FILE)

Do tập dữ liệu có nhiều điểm ngoại lai nên tiến hành thay thế các giá trị 0 thành giá trị mean của các mẫu dữ liệu cùng nhãn lớp

In [5]:
def class_fill_na(column):
    not_null_val = df[df[column].notnull()][[column, 'Outcome']]
    median_value = not_null_val.groupby(['Outcome'])[[column]].median().reset_index()
    df.loc[(df['Outcome'] == 0) & (df[column].isnull()), column] = median_value[column][0]
    df.loc[(df['Outcome'] == 1) & (df[column].isnull()), column] = median_value[column][1]

In [6]:
non_zero_columns = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for column in non_zero_columns:
    class_fill_na(column)

In [7]:
df.isnull().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

In [9]:
NAN_TO_MEDIAN_FILE = PROCESSED_DATA_DIR / 'nan_to_median.csv'
df.to_csv(NAN_TO_MEDIAN_FILE, index= False)